In [ ]:
#!pip3 install openpyxl
#!pip3 install pandas
!pip3 install nltk

     |################################| 1.5 MB 21.8 MB/s eta 0:00:01
     |################################| 748 kB 33.6 MB/s eta 0:00:01
  Attempting uninstall: regex
    Found existing installation: regex 2020.11.13
    Uninstalling regex-2020.11.13:
      Successfully uninstalled regex-2020.11.13
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [ ]:
import json
with open("m_data/echr_sample.json", "r") as f:
    sample = json.load(f)

In [ ]:
from data_creation import ECHRData
echr = ECHRData()

In [ ]:
cases = echr.get_data_as_sentences()
appeal_sentences = []
for case in cases:
    case_set = set(case)
    for sentence in case_set:
        if "appeal" in sentence:
            appeal_sentences.append(sentence.replace('\xa0', ' '))

In [ ]:
import re
date = "(?:\d{1,2}) (?:\w+) (?:\d{2,4})"
event = "".join([
        "(quashed|upheld|allowed|dismissed) ",
        "(?:the|that|an) (?:(?:applicant’s|applicant party’s) )?",
        "(judgment|decision|application|appeal|conviction)",
        "(?: of ({date}))?"
        "(?: by the applicant)?"
        "( on appeal)?"
        ]).format(date=date)
appeal_regex = re.compile(
    "".join([
        "On ({date}) the ([\w* ]*)(?:court|Court) ",
        "{event}",
        "(?: and {event})*"
        "\."
    ])      
    .format(date=date, event=event))
appeal_regex3 = re.compile("On {date} the ([\w* ]*)(applicant|applicants|lawyer) \d* (appealed|lodged).*")

In [ ]:
match_count = 0
matched = []
for sentence in appeal_sentences:
    m2 = appeal_regex.match(sentence)
    if m2 is not None: #and m2 is None:
        matched.append((sentence, m2.groups()))
        #print(m.groups())
        match_count += 1
print(match_count)

658


In [ ]:
echr_knowledge = []
_id = 0
for sentence, groups in matched:
    point_dict = dict()
    point_dict["id"] = _id
    point_dict["sentence"] = sentence
    point_dict["date"] = groups[0]
    if groups[1] is not None and groups[1] != '':
        point_dict["court_type_or_name"] = groups[1] + "Court"
    i = 2
    actions = []
    while i < len(groups) and groups[i] is not None:
        action = dict()
        action["action"] = groups[i]
        action["object"] = groups[i + 1]
        if groups[i + 2] is not None:
            action["object_date"] = groups[i + 2]
        if groups[i + 3] is not None:
            action["appeal"] = True
        i += 4
        actions.append(action)
        
    point_dict["actions"] = actions
    
    echr_knowledge.append(point_dict)
    _id += 1

In [ ]:
import json
with open("echr_knowledge_appeal.json", "w") as f:
    json.dump(echr_knowledge, f)

In [ ]:
for point in echr_knowledge:
    free_var = "a"
    logical_form = ""
    logical_form += "Date({})".format(point["date"])
    if "court_type_or_name" in point:
        logical_form += "^Court({})".format(point["court_type_or_name"])
    else:
        logical_form += "^Court()"
    for action in point["actions"]:
        if action["action"] == "upheld":
            logical_form += "^Uphold(\{})".format(free_var)
            logical_form += "^UpheldObject(\{}, {})".format(free_var, action["object"])
            if "appeal" in action:
                logical_form += "^UpheldOnAppeal(\{})".format(free_var)
        elif action["action"] == "quashed":
            logical_form += "^Quash(\{})".format(free_var)
            logical_form += "^QuashedObject(\{}, {})".format(free_var, action["object"])
            if "appeal" in action:
                logical_form += "^QuashedOnAppeal(\{})".format(free_var)
        elif action["action"] == "allowed":
            logical_form += "^Allow(\{})".format(free_var)
            logical_form += "^AllowedObject(\{}, {})".format(free_var, action["object"])
            if "appeal" in action:
                logical_form += "^AllowedOnAppeal(\{})".format(free_var)
        elif action["action"] == "dismissed":
            logical_form += "^Dismiss(\{})".format(free_var)
            logical_form += "^DismissedObject(\{}, {})".format(free_var, action["object"])
            if "appeal" in action:
                logical_form += "^DismissedOnAppeal(\{})".format(free_var)
        free_var = chr(ord(free_var) + 1)
    point["fol_form"] = logical_form

In [ ]:
print(echr_knowledge[421])

{'id': 421, 'sentence': 'On 30 October 2003 the court dismissed the applicant’s appeal.', 'date': '30 October 2003', 'actions': [{'action': 'dismissed', 'object': 'appeal'}], 'fol_form': 'Date(30 October 2003)^Court()^Dismiss(\\a)^DismissedObject(\\a, appeal)'}


In [ ]:
cases = echr.get_data_as_sentences()
refer_sentences = []
for case in cases:
    case_set = set(case)
    for sentence in case_set:
        if "was referred to" in sentence:
            refer_sentences.append(sentence.replace('\xa0', ' '))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
len(refer_sentences)

1458

In [ ]:
refer_sentences[:10]

['After the entry into force of Protocol No. 11 on 1 November 1998 and in accordance with the provisions of Article 5 § 5 thereof, the case was referred to the Grand Chamber of the Court.',
 'The case was referred to the Court, as established under former Article 19 of the Convention, by the European Commission of Human Rights (“the Commission”) on 24 September 1999, within the three-month period laid down by former Articles 32 § 1 and 47 of the Convention.',
 'The case was referred to the Court by the European Commission of Human Rights ("the Commission") and by the Italian Government ("the Government") on 29 May 1995, within the three-month period laid down by Article 32 para. 1 and Article 47 of the Convention (art. 32-1, art. 47).',
 'The present case was referred to the Court, within the period of three months laid down by Articles 32 para. 1 and 47 (art. 32-1, art. 47) of the Convention for the Protection of Human Rights and Fundamental Freedoms ("the Convention"), by the Europea

In [ ]:
date = "(?:\d{1,2}) (?:\w+) (?:\d{2,4})"
conj = "(?:(?:and )|(, ))"
words = "[(\w|,|.|\"|\(|\)))* ]*"
no_by_on = "([^\n(, by )( on )]*)"
no_on_within = "([^\n( within )( on )]*)"
words = ".*"
by = "(?: by (?P<by>{words}))".format(words=words)
by2 = "(?: by (?P<by>{words}))".format(words=no_on_within)
on = "(?: on (?P<on>{date}))".format(date=date)
within="(?: within (?P<within>{words}))?".format(words=words)
within2="(?: within (?P<within>{words}))?".format(words=no_by_on)
by_on_within = "(?:{conj}?(?:{by}|{on}|{within}))*".format(by=by, on=on, within=within, conj=conj)
by_on_within2 = "(?:{conj}?(?:{by}|{on}|{within}))*".format(by=by2, on=on, within=within2, conj=conj)

refer_regex = re.compile(
    "".join([
        "The (?:present )?(?:case|application)((?: of)? {words})? was referred to the (?P<courtname>[\w* ]*)(?:court|Court){by_on_within}\.",
    ])
    .format(words=words, by_on_within=by_on_within2)
)

refer_regex1 = re.compile(
    "".join([
        "The (?:present )?(?:case|application)((?: of)? {words})? was referred to the (?P<courtname>[\w* ]*)(?:court|Court){by_on_within}\.",
    ])
    .format(words=words, by_on_within=by_on_within)
)

refer_regex3 = re.compile(
    "".join([
        "The (?:present )?(?:case|application)((?: of)? {words})? was referred to the (?P<courtname>[\w* ]*)(?:court|Court){by_on_within}.*",
    ])
    .format(words=words, by_on_within=by_on_within)
)


#refer_regex = re.compile()

refer_regex2 = re.compile(
    "".join([
        "The (present )?(case|application).*",
    ])    
)

In [ ]:
match_count = 0
matched = []
for sentence in refer_sentences:
    m = refer_regex.match(sentence)
    m2 = refer_regex1.match(sentence)
    if m2 is not None and m is None:
        matched.append((sentence, m2.groupdict()))
        #print(sentence)
        #print(m.groups())
        match_count += 1
print(match_count)

549


In [ ]:
words = "[(\w|,|.|\"|\(|\)))* ]*"
conj = "(?:( and|,))"
no_by_on = ".*?(?=, by | on )"
no_on_within = ".*?(?= within | on )"
by2 = "(?: by (?P<by>{words}))".format(words=no_on_within)
within2="(?: within (?P<within>{words}))".format(words=no_by_on)
by_on_within2 = "(?P<s>({conj}?(?:{by}|{on}|{within}))*)".format(by=by2, on=on, within=within2, conj=conj)
refer_regex = re.compile(
    "".join([
        "The case was referred to the Court{by_on_within}.*",
    ])
    .format(words=words, by_on_within=by_on_within2)
)

m = refer_regex.match("The case was referred to the Court by the European Commission of Human Rights (“the Commission”) on 28 October 1996 and by the Italian Parliament, within the three-month period laid down by Article 32 § 1 and Article 47 of the Convention.")
print(m.groups())
print("The case was referred to the Court by the European Commission of Human Rights (“the Commission”) on 28 October 1996, within the three-month period laid down by Article 32 § 1 and Article 47 of the Convention.")

(' by the European Commission of Human Rights (“the Commission”) on 28 October 1996 and by the Italian Parliament,', ' and by the Italian Parliament,', ' and', 'the Italian Parliament,', '28 October 1996', None)
The case was referred to the Court by the European Commission of Human Rights (“the Commission”) on 28 October 1996, within the three-month period laid down by Article 32 § 1 and Article 47 of the Convention.


In [ ]:
re.compile(".*?(?= within | on )").match("x on y")

<_sre.SRE_Match object; span=(0, 1), match='x'>

In [ ]:
matched[10:20]

[('The case was referred to the Court by the European Commission of Human Rights (“the Commission”) on 28 October 1996, within the three-month period laid down by Article 32 § 1 and Article 47 of the Convention.',
  {'courtname': '',
   'by': 'the European Commission of Human Rights (“the Commission”) on 28 October 1996, within the three-month period laid down by Article 32 § 1 and Article 47 of the Convention',
   'on': None,
   'within': None}),
 ('The case was referred to the Court by the French Government (“the Government”) on 14 April 1997, within the three-month period laid down by Article 32 § 1 and Article 47 of the Convention.',
  {'courtname': '',
   'by': 'the French Government (“the Government”) on 14 April 1997, within the three-month period laid down by Article 32 § 1 and Article 47 of the Convention',
   'on': None,
   'within': None}),
 ('The case was referred to the Court by the European Commission of Human Rights ("the Commission") on 23 October 1987, within the three

In [ ]:
s = "ab"

In [ ]:
s[::-1]

'ba'

In [ ]:
cases = echr.get_data_as_sentences()
words = "[(?:\w|,|.|\"|\(|\)))* ]*"

passive_voice_regex = re.compile(
    "".join([
        "{words} (?:(?:was)|(?:were)) (\w*d) {words}."
    ])
    .format(words=words)
)

passive_voice_matches = []
verbs = dict()
match_count = 0
for case in cases:
    for sentence in case:
        m = passive_voice_regex.match(sentence)
        if m is not None:
            passive_voice_matches.append((sentence, m.groups()))
            match_count += 1
            verb = m.groups()[0]
            if verb in verbs:
                verbs[verb] += 1
            else:
                verbs[verb] = 1
print(match_count)

227141


In [ ]:
sorted(verbs.items(), key = lambda x: x[1], reverse=True)[:15]

[('represented', 12647),
 ('held', 4844),
 ('arrested', 4051),
 ('detained', 3678),
 ('communicated', 3614),
 ('transferred', 3428),
 ('based', 3413),
 ('informed', 3321),
 ('found', 3286),
 ('examined', 3259),
 ('granted', 3012),
 ('adjourned', 2950),
 ('required', 2770),
 ('entitled', 2743),
 ('served', 2688)]

In [ ]:
cases = echr.get_data_as_sentences()
from nltk.tokenize import word_tokenize
short_sentences = []

for case in cases:
    for sentence in case:
        words = word_tokenize(sentence)
        for p in [".", ":", ";"]:
            if p in words:
                words.remove(p)
        if 1 < len(words) < 6:
            short_sentences.append(words)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
len(short_sentences)

NameError: name 'short_sentences' is not defined

In [ ]:
short_sentences[:5]

[['1', '.'], ['2', '.'], ['3', '.'], ['4', '.'], ['THE', 'FACTS']]

In [ ]:
short_sentences_filtered = [sentence for sentence in short_sentences if len(sentence) > 2]

In [ ]:
short_sentences_filtered2 = []
for sentence in short_sentences_filtered[:100000]:
    doc = nlp(" ".join(sentence))
    if any([token.pos_ == "VERB" for token in doc]):
        short_sentences_filtered2.append(sentence)

In [ ]:
s = set([" ".join(sentence) for sentence in short_sentences_filtered2])

In [ ]:
list(s)[20:40]

['Principles established by the court',
 'Circulus in demonstrando !',
 'The relevant parts read :',
 'Paget-Lewis is convicted of manslaughter',
 'While there Ģ.Č .',
 'held periodic surgeries . ”',
 'It consequently rejects them .',
 'turned to Mr V.V .',
 'As regards Ms Slavojka Kandić',
 'No reply ensued .',
 'and remitted the case .',
 '19 measuring 23.6 sq .',
 'Bringing of an action',
 'It thus rules accordingly .',
 'We immediately paid .',
 'The involvement includes :',
 '2943/6 were still pending .',
 'And the pressure continued .',
 'Nedir bu ilke ?',
 'd. Renewed proceedings before the']

In [ ]:
!pip3 install spacy
!export LC_ALL=C.UTF-8
!export LANG=C.UTF-8
!python -m spacy download en_core_web_sm

/usr/bin/python: No module named spacy


In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
x = [".", "basdas", "adsas"]
x.remove(",")
x

ValueError: list.remove(x): x not in list

In [ ]:
cases = echr.get_data_as_texts()
holds_cases = []
for case in cases:
    if "for these reasons" in case.lower() and "holds that" in case.lower():
        holds_cases.append(case.replace('\xa0', ' '))

holds_sentences = []
for holds_case in holds_cases:
    case = holds_case.lower().replace("\n", " ")
    sentences = echr.tokenizer.tokenize(case)
    for sentence in sentences:
        if "for these reasons" in sentence or "holds that" in sentence:
            holds_sentences.append(sentence)

In [ ]:
count = 0
for case in cases:
    if "for these reasons" not in case.lower():
        print(case)

In [ ]:
len(cases)

In [ ]:
declares_the_application = set()
declares_the_application_count = 0
for sentence in holds_sentences:
    s = re.search("(declares the application inadmissible.*)", sentence)
    if s:
        declares_the_application.add(s.groups()[0])
        declares_the_application_count += 1